### Import Required Libraries and Set Up Environment Variables

In [6]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from requests.adapters import HTTPAdapter

In [7]:
# Set environment variables from the .env in the local environment
load_dotenv()
nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [9]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies"ANDtype_of_material:"Review"ANDheadline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"
# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
long_url = f"{url}fq={filter_query}&sort={sort}&begin_date={begin_date}&end_date={end_date}&fl={field_list}&api-key={nyt_api_key}"

In [10]:
# Create an empty list to store the reviews
content_list = []
pages = 20
retry_after = 12
# loop through pages 0-19
    # create query with a page number
    # API results show 10 articles at a time
    # Make a "GET" request and retrieve the JSON    
    # Add a twelve second interval between queries to stay within API query limits
for page in range(pages):
    query_url = f"{long_url}&page={page}"
    print(f'Page#:{page} - {query_url}')
    data_json = requests.get(query_url).json()
    resp_docs = data_json["response"]["docs"]
    content_list.append(resp_docs)
    time.sleep(retry_after)


Page#:0 - https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies"ANDtype_of_material:"Review"ANDheadline:"love"&sort=newest&begin_date=20130101&end_date=20230531&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&api-key=wYSOBlD014ZsMNZlF8gv9YCmwQeImKFF&page=0
Page#:1 - https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies"ANDtype_of_material:"Review"ANDheadline:"love"&sort=newest&begin_date=20130101&end_date=20230531&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&api-key=wYSOBlD014ZsMNZlF8gv9YCmwQeImKFF&page=1
Page#:2 - https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies"ANDtype_of_material:"Review"ANDheadline:"love"&sort=newest&begin_date=20130101&end_date=20230531&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&api-key=wYSOBlD014ZsMNZlF8gv9YCmwQeImKFF&page=2
Page#:3 - https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_n

In [ ]:
 # Try and save the reviews to the reviews_list
        # loop through the reviews["response"]["docs"] and append each review to the list
        # Print the page that was just retrieved
        # Print the page number that had no results then break from the loop
reviews_list = [content for content_info in content_list for content in content_info]
        

In [12]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/31/movies/reality-review.html",
        "snippet": "A new docudrama starring Sydney Sweeney as Reality Winner is gripping, even as it strips a true story of its political context.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018Reality\u2019 Review: An Unusual Suspect",
            "kicker": "Critic\u2019s Pick",
            "content_kicker": null,
            "print_headline": "Examining A Most Unusual Suspect",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "Reality (2023) (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "

In [13]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
movie_reviews_df = pd.json_normalize(reviews_list)
movie_reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/31/movies/real...,A new docudrama starring Sydney Sweeney as Rea...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-31T23:26:13+0000,499,‘Reality’ Review: An Unusual Suspect,Critic’s Pick,None,Examining A Most Unusual Suspect,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
1,https://www.nytimes.com/2023/05/31/movies/docu...,This month’s selections include a look at a po...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-05-31T20:34:03+0000,1155,Stream These Three Great Documentaries,None,None,None,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
2,https://www.nytimes.com/2023/05/31/movies/the-...,"An interview with the actors, who have big cla...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-31T18:53:03+0000,812,Daveed Diggs and Awkwafina Discuss ‘The Little...,None,None,Insight From a Gull and a Crab,None,None,None,By Leah Greenblatt,"[{'firstname': 'Leah', 'middlename': None, 'la...",None
3,https://www.nytimes.com/2023/05/31/movies/gret...,She’s known for playing offbeat characters in ...,The New York Times,"[{'name': 'persons', 'value': 'Lee, Greta', 'r...",2023-05-31T08:56:19+0000,1888,How to Be a Star by Not Giving Your All,None,None,How to Be a Star by Holding Back,None,None,None,By Robert Ito,"[{'firstname': 'Robert', 'middlename': None, '...",None
4,https://www.nytimes.com/2023/05/29/movies/alan...,The composer talked revisiting “The Little Mer...,The New York Times,"[{'name': 'persons', 'value': 'Menken, Alan', ...",2023-05-29T09:01:13+0000,1073,Alan Menken on ‘The Little Mermaid’ and the Mi...,None,None,Learning to Embrace His Disney Fame,None,None,None,By Sarah Bahr,"[{'firstname': 'Sarah', 'middlename': None, 'l...",None


In [25]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
movie_reviews_df["title"] = movie_reviews_df['headline.main'].str.extract(r"\u2018(.*?)\u2019.*Review")


In [26]:
movie_reviews_df[['headline.main', 'title']].head()


,headline.main,title
0,‘Reality’ Review: An Unusual Suspect,Reality
1,Stream These Three Great Documentaries,NaN
2,Daveed Diggs and Awkwafina Discuss ‘The Little...,NaN
3,How to Be a Star by Not Giving Your All,NaN
4,Alan Menken on ‘The Little Mermaid’ and the Mi...,NaN


In [27]:
movie_reviews_df['title'].describe()

count         109
unique        109
top       Reality
freq            1
Name: title, dtype: object

In [28]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
movie_reviews_copy = movie_reviews_df.copy()
movie_reviews_copy["keywords"] = movie_reviews_copy["keywords"].apply(extract_keywords)
movie_reviews_copy.head(5)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/31/movies/real...,A new docudrama starring Sydney Sweeney as Rea...,The New York Times,subject: Movies;creative_works: Reality (2023)...,2023-05-31T23:26:13+0000,499,‘Reality’ Review: An Unusual Suspect,Critic’s Pick,None,Examining A Most Unusual Suspect,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Reality
1,https://www.nytimes.com/2023/05/31/movies/docu...,This month’s selections include a look at a po...,The New York Times,subject: Documentary Films and Programs;creati...,2023-05-31T20:34:03+0000,1155,Stream These Three Great Documentaries,None,None,None,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None,NaN
2,https://www.nytimes.com/2023/05/31/movies/the-...,"An interview with the actors, who have big cla...",The New York Times,"subject: Movies;persons: Diggs, Daveed;persons...",2023-05-31T18:53:03+0000,812,Daveed Diggs and Awkwafina Discuss ‘The Little...,None,None,Insight From a Gull and a Crab,None,None,None,By Leah Greenblatt,"[{'firstname': 'Leah', 'middlename': None, 'la...",None,NaN
3,https://www.nytimes.com/2023/05/31/movies/gret...,She’s known for playing offbeat characters in ...,The New York Times,"persons: Lee, Greta;subject: Actors and Actres...",2023-05-31T08:56:19+0000,1888,How to Be a Star by Not Giving Your All,None,None,How to Be a Star by Holding Back,None,None,None,By Robert Ito,"[{'firstname': 'Robert', 'middlename': None, '...",None,NaN
4,https://www.nytimes.com/2023/05/29/movies/alan...,The composer talked revisiting “The Little Mer...,The New York Times,"persons: Menken, Alan;persons: Miranda, Lin-Ma...",2023-05-29T09:01:13+0000,1073,Alan Menken on ‘The Little Mermaid’ and the Mi...,None,None,Learning to Embrace His Disney Fame,None,None,None,By Sarah Bahr,"[{'firstname': 'Sarah', 'middlename': None, 'l...",None,NaN


In [29]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
movie_titles = movie_reviews_copy["title"].dropna().to_list()
movie_titles

['Reality',
 'The Machine',
 'You Hurt My Feelings',
 'The Wind & the Reckoning',
 'We Might as Well Be Dead',
 'Will-o',
 'Unclenching the Fists',
 'White Balls on Walls',
 'The Wrath of Becky',
 'Being Mary Tyler Moore',
 'Kandahar',
 'About My Father',
 'The Attachment Diaries',
 'Influencer',
 'The Little Mermaid',
 'Victim/Suspect',
 'Fast X',
 'The Taking',
 'Stay Awake',
 'Master Gardener',
 'Butterfly Vision',
 'Museum of the Revolution',
 'The Night of the 12th',
 'White Men Can',
 'White Building',
 'Sanctuary',
 'Come Out Fighting',
 'Moon Garden',
 'The Cow Who Sang a Song Into the Future',
 'Queenmaker',
 'Anna Nicole Smith: You Don',
 'Giving Birth to a Butterfly',
 'Hypnotic',
 'Crater',
 'The Mother',
 'Book Club: The Next Chapter',
 'The Starling Girl',
 'L',
 'It Ain',
 'Still: A Michael J. Fox Movie',
 'Unfinished Business',
 'Monica',
 'BlackBerry',
 'Fool',
 'Dealing With Dad',
 'The Melt Goes On Forever',
 'Queens of the Qing Dynasty',
 'You Can Live Forever',
 'S

### Access The Movie Database API

In [30]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [43]:
# Create an empty list to store the results
# Create a request counter to sleep the requests after a multiple
# of 50 requests
# Loop through the titles
# Check if we need to sleep before making a request
# Add 1 to the request counter    
# Perform a "GET" request for The Movie Database
# Include a try clause to search for the full movie details.
# Use the except clause to print out a statement if a movie
# is not found.
# Get movie id
# Make a request for a the full movie details
# Execute "GET" request with url
# Extract the genre names into a list
# Extract the spoken_languages' English name into a list
# Extract the production_countries' name into a list
# Add the relevant data to a dictionary and
# append it to the tmdb_movies_list list
tmdb_movies_list = []
counter = 1
for title in movie_titles:
    try:
        query_url = url + title + tmdb_key_string
        response = requests.get(query_url).json()
        time.sleep(1)
        counter += 1
        if counter % 50 == 0:
            time.sleep(10)
        movie_id = response["results"][0]["id"]
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?{tmdb_key_string}"
        movie_response = requests.get(movie_url).json()
        genre_list = [genre["name"] for genre in movie_response["genres"]]
        spoken_languages = [language["english_name"] for language in movie_response["spoken_languages"]]
        production_countries = [country["name"] for country in movie_response["production_countries"]]
        movie_dict = {
            "title": title,
            "genre": genre_list,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        }
        tmdb_movies_list.append(movie_dict)
    except:
        continue


In [ ]:
# Print out the title that was found
for k in tmdb_movies_list:
    print(f"Found Movie: {k['title']}")


The Plains
Found Movie: Reality
Found Movie: The Machine
Found Movie: You Hurt My Feelings
Found Movie: The Wind & the Reckoning
Found Movie: We Might as Well Be Dead
Found Movie: Will-o
Found Movie: Unclenching the Fists
Found Movie: White Balls on Walls
Found Movie: The Wrath of Becky
Found Movie: Being Mary Tyler Moore
Found Movie: Kandahar
Found Movie: About My Father
Found Movie: The Attachment Diaries
Found Movie: Influencer
Found Movie: The Little Mermaid
Found Movie: Victim/Suspect
Found Movie: Fast X
Found Movie: The Taking
Found Movie: Stay Awake
Found Movie: Master Gardener
Found Movie: Butterfly Vision
Found Movie: Museum of the Revolution
Found Movie: The Night of the 12th
Found Movie: White Men Can
Found Movie: White Building
Found Movie: Sanctuary
Found Movie: Come Out Fighting
Found Movie: Moon Garden
Found Movie: The Cow Who Sang a Song Into the Future
Found Movie: Queenmaker
Found Movie: Anna Nicole Smith: You Don
Found Movie: Giving Birth to a Butterfly
Found Movie: 

In [46]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))


[
    {
        "title": "Reality",
        "genre": [
            "Drama",
            "Thriller"
        ],
        "spoken_languages": [
            "English"
        ],
        "production_countries": [
            "United States of America",
            "United Kingdom"
        ]
    },
    {
        "title": "The Machine",
        "genre": [
            "Comedy",
            "Action",
            "Crime"
        ],
        "spoken_languages": [
            "Spanish",
            "English"
        ],
        "production_countries": [
            "United States of America"
        ]
    },
    {
        "title": "You Hurt My Feelings",
        "genre": [
            "Comedy",
            "Drama",
            "Romance"
        ],
        "spoken_languages": [
            "English",
            "Spanish"
        ],
        "production_countries": [
            "United States of America"
        ]
    },
    {
        "title": "The Wind & the Reckoning",
        "genre": [
           

In [47]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_df.head()

,title,genre,spoken_languages,production_countries
0,Reality,"[Drama, Thriller]",[English],"[United States of America, United Kingdom]"
1,The Machine,"[Comedy, Action, Crime]","[Spanish, English]",[United States of America]
2,You Hurt My Feelings,"[Comedy, Drama, Romance]","[English, Spanish]",[United States of America]
3,The Wind & the Reckoning,[Drama],[Japanese],[Japan]
4,We Might as Well Be Dead,"[Comedy, Drama]","[German, Polish]","[Germany, Romania]"


### Merge and Clean the Data for Export

In [60]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_moive_df = movie_reviews_copy.merge(tmdb_movies_df, on="title", how="inner")
merged_moive_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,genre,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/31/movies/real...,A new docudrama starring Sydney Sweeney as Rea...,The New York Times,subject: Movies;creative_works: Reality (2023)...,2023-05-31T23:26:13+0000,499,‘Reality’ Review: An Unusual Suspect,Critic’s Pick,None,Examining A Most Unusual Suspect,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Reality,"[Drama, Thriller]",[English],"[United States of America, United Kingdom]"
1,https://www.nytimes.com/2023/05/26/movies/the-...,In a movie extrapolated from one of his stand-...,The New York Times,subject: Movies;creative_works: The Machine (M...,2023-05-26T22:33:26+0000,318,‘The Machine’ Review: A Hard-Partying Comedian...,None,None,"Gone to Seed, With Some Seedy Company",None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Machine,"[Comedy, Action, Crime]","[Spanish, English]",[United States of America]
2,https://www.nytimes.com/2023/05/25/movies/you-...,The director Nicole Holofcener’s characters ar...,The New York Times,"subject: Movies;persons: Holofcener, Nicole;pe...",2023-05-25T18:47:33+0000,1039,‘You Hurt My Feelings’ Review: She Can’t Handl...,Critic’s Pick,None,Welcome to the Little White Lie Club,None,None,None,By Wesley Morris,"[{'firstname': 'Wesley', 'middlename': None, '...",None,You Hurt My Feelings,"[Comedy, Drama, Romance]","[English, Spanish]",[United States of America]
3,https://www.nytimes.com/2023/05/25/movies/the-...,A docudrama follows a family fighting to stay ...,The New York Times,"subject: Movies;persons: Lee, Jason Scott;pers...",2023-05-25T18:25:49+0000,273,‘The Wind & the Reckoning’ Review: A Hawaiian ...,None,None,The Wind &amp; the Reckoning,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None,The Wind & the Reckoning,[Drama],[Japanese],[Japan]
4,https://www.nytimes.com/2023/05/25/movies/we-m...,A single mother fights for her place in a dyst...,The New York Times,"subject: Movies;persons: Iacob, Ioana;creative...",2023-05-25T18:24:47+0000,294,‘We Might as Well Be Dead’ Review: Housing by ...,None,None,We Might as Well Be Dead,None,None,None,By Teo Bugbee,"[{'firstname': 'Teo', 'middlename': None, 'las...",None,We Might as Well Be Dead,"[Comedy, Drama]","[German, Polish]","[Germany, Romania]"


In [61]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
# Create a list of characters to remove
# Loop through the list of columns to fix
    # Convert the column to type 'str'
    # Loop through characters to remove
# Display the fixed DataFrame
merged_moive_df = merged_moive_df.astype(str)
columns_to_fix = ["genre", "spoken_languages", "production_countries"]
chars_to_remove = ["[", "]", "'"]
for column in columns_to_fix:
    for char in chars_to_remove:
        merged_moive_df[column] = merged_moive_df[column].str.replace(char, "")
merged_moive_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,genre,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/31/movies/real...,A new docudrama starring Sydney Sweeney as Rea...,The New York Times,subject: Movies;creative_works: Reality (2023)...,2023-05-31T23:26:13+0000,499,‘Reality’ Review: An Unusual Suspect,Critic’s Pick,None,Examining A Most Unusual Suspect,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Reality,"Drama, Thriller",English,"United States of America, United Kingdom"
1,https://www.nytimes.com/2023/05/26/movies/the-...,In a movie extrapolated from one of his stand-...,The New York Times,subject: Movies;creative_works: The Machine (M...,2023-05-26T22:33:26+0000,318,‘The Machine’ Review: A Hard-Partying Comedian...,None,None,"Gone to Seed, With Some Seedy Company",None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Machine,"Comedy, Action, Crime","Spanish, English",United States of America
2,https://www.nytimes.com/2023/05/25/movies/you-...,The director Nicole Holofcener’s characters ar...,The New York Times,"subject: Movies;persons: Holofcener, Nicole;pe...",2023-05-25T18:47:33+0000,1039,‘You Hurt My Feelings’ Review: She Can’t Handl...,Critic’s Pick,None,Welcome to the Little White Lie Club,None,None,None,By Wesley Morris,"[{'firstname': 'Wesley', 'middlename': None, '...",None,You Hurt My Feelings,"Comedy, Drama, Romance","English, Spanish",United States of America
3,https://www.nytimes.com/2023/05/25/movies/the-...,A docudrama follows a family fighting to stay ...,The New York Times,"subject: Movies;persons: Lee, Jason Scott;pers...",2023-05-25T18:25:49+0000,273,‘The Wind & the Reckoning’ Review: A Hawaiian ...,None,None,The Wind &amp; the Reckoning,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None,The Wind & the Reckoning,Drama,Japanese,Japan
4,https://www.nytimes.com/2023/05/25/movies/we-m...,A single mother fights for her place in a dyst...,The New York Times,"subject: Movies;persons: Iacob, Ioana;creative...",2023-05-25T18:24:47+0000,294,‘We Might as Well Be Dead’ Review: Housing by ...,None,None,We Might as Well Be Dead,None,None,None,By Teo Bugbee,"[{'firstname': 'Teo', 'middlename': None, 'las...",None,We Might as Well Be Dead,"Comedy, Drama","German, Polish","Germany, Romania"


In [63]:
# Drop "byline.person" column
merged_moive_df = merged_moive_df.drop(columns=["byline.person"])
merged_moive_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization,title,genre,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/31/movies/real...,A new docudrama starring Sydney Sweeney as Rea...,The New York Times,subject: Movies;creative_works: Reality (2023)...,2023-05-31T23:26:13+0000,499,‘Reality’ Review: An Unusual Suspect,Critic’s Pick,None,Examining A Most Unusual Suspect,None,None,None,By Amy Nicholson,None,Reality,"Drama, Thriller",English,"United States of America, United Kingdom"
1,https://www.nytimes.com/2023/05/26/movies/the-...,In a movie extrapolated from one of his stand-...,The New York Times,subject: Movies;creative_works: The Machine (M...,2023-05-26T22:33:26+0000,318,‘The Machine’ Review: A Hard-Partying Comedian...,None,None,"Gone to Seed, With Some Seedy Company",None,None,None,By Glenn Kenny,None,The Machine,"Comedy, Action, Crime","Spanish, English",United States of America
2,https://www.nytimes.com/2023/05/25/movies/you-...,The director Nicole Holofcener’s characters ar...,The New York Times,"subject: Movies;persons: Holofcener, Nicole;pe...",2023-05-25T18:47:33+0000,1039,‘You Hurt My Feelings’ Review: She Can’t Handl...,Critic’s Pick,None,Welcome to the Little White Lie Club,None,None,None,By Wesley Morris,None,You Hurt My Feelings,"Comedy, Drama, Romance","English, Spanish",United States of America
3,https://www.nytimes.com/2023/05/25/movies/the-...,A docudrama follows a family fighting to stay ...,The New York Times,"subject: Movies;persons: Lee, Jason Scott;pers...",2023-05-25T18:25:49+0000,273,‘The Wind & the Reckoning’ Review: A Hawaiian ...,None,None,The Wind &amp; the Reckoning,None,None,None,By Concepción de León,None,The Wind & the Reckoning,Drama,Japanese,Japan
4,https://www.nytimes.com/2023/05/25/movies/we-m...,A single mother fights for her place in a dyst...,The New York Times,"subject: Movies;persons: Iacob, Ioana;creative...",2023-05-25T18:24:47+0000,294,‘We Might as Well Be Dead’ Review: Housing by ...,None,None,We Might as Well Be Dead,None,None,None,By Teo Bugbee,None,We Might as Well Be Dead,"Comedy, Drama","German, Polish","Germany, Romania"


In [64]:
# Delete duplicate rows and reset index
merged_moive_df = merged_moive_df.drop_duplicates().reset_index(drop=True)
merged_moive_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization,title,genre,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/31/movies/real...,A new docudrama starring Sydney Sweeney as Rea...,The New York Times,subject: Movies;creative_works: Reality (2023)...,2023-05-31T23:26:13+0000,499,‘Reality’ Review: An Unusual Suspect,Critic’s Pick,None,Examining A Most Unusual Suspect,None,None,None,By Amy Nicholson,None,Reality,"Drama, Thriller",English,"United States of America, United Kingdom"
1,https://www.nytimes.com/2023/05/26/movies/the-...,In a movie extrapolated from one of his stand-...,The New York Times,subject: Movies;creative_works: The Machine (M...,2023-05-26T22:33:26+0000,318,‘The Machine’ Review: A Hard-Partying Comedian...,None,None,"Gone to Seed, With Some Seedy Company",None,None,None,By Glenn Kenny,None,The Machine,"Comedy, Action, Crime","Spanish, English",United States of America
2,https://www.nytimes.com/2023/05/25/movies/you-...,The director Nicole Holofcener’s characters ar...,The New York Times,"subject: Movies;persons: Holofcener, Nicole;pe...",2023-05-25T18:47:33+0000,1039,‘You Hurt My Feelings’ Review: She Can’t Handl...,Critic’s Pick,None,Welcome to the Little White Lie Club,None,None,None,By Wesley Morris,None,You Hurt My Feelings,"Comedy, Drama, Romance","English, Spanish",United States of America
3,https://www.nytimes.com/2023/05/25/movies/the-...,A docudrama follows a family fighting to stay ...,The New York Times,"subject: Movies;persons: Lee, Jason Scott;pers...",2023-05-25T18:25:49+0000,273,‘The Wind & the Reckoning’ Review: A Hawaiian ...,None,None,The Wind &amp; the Reckoning,None,None,None,By Concepción de León,None,The Wind & the Reckoning,Drama,Japanese,Japan
4,https://www.nytimes.com/2023/05/25/movies/we-m...,A single mother fights for her place in a dyst...,The New York Times,"subject: Movies;persons: Iacob, Ioana;creative...",2023-05-25T18:24:47+0000,294,‘We Might as Well Be Dead’ Review: Housing by ...,None,None,We Might as Well Be Dead,None,None,None,By Teo Bugbee,None,We Might as Well Be Dead,"Comedy, Drama","German, Polish","Germany, Romania"


In [65]:
# Export data to CSV without the index
merged_moive_df.to_csv("output/nyt_movie_reviews.csv", index=False)